In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from method import IV
from simulator import Simulator
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec
import pandas as pd
import numpy as np
import quantities as pq
import seaborn as sns
from tools import savefig, fix_figure, coef_var, corrcoef, despine, set_style
import copy
from IPython.display import display
import statsmodels.api as sm

ImportError: cannot import name 'plot_xcorr'

# Add hit rate

In [15]:
trials = [5000, 10000, 15000, 20000, 25000, 30000]
sim_id = [1, 2, 3]
# path = 'results/orig_analysis/'
path = 'results/'
winsize = 4

for sim in sim_id:
    data = np.load('results/params_{}.npz'.format(sim))['data'][()]

    spiketrains = {s['sender']: {'pop': pop, 'times': s['times']}
                   for pop in ['in', 'ex']
                   for s in data['spiketrains'][pop]}

    s = data['epoch']['times'][:trial]
    assert len(s) == trial
    latency = data['params']['delay'] + data['params']['tau_syn_ex']
    
    for trial in trials:
        csv_path = 'results/params_{}_analyse_{}.csv'.format(sim, trial)
        df = pd.read_csv(csv_path)
        hit_rate = {}
        for idx, row in df.iterrows():
            if row.source in hit_rate.keys():
                df.loc[idx,'hit_rate'] = hit_rate[row.source]
                continue
            t1 = spiketrains[row.source]['times']
            t2 = spiketrains[row.target]['times']
            iv = IV(t1, t2, s, winsize, latency)
            df.loc[idx,'hit_rate'] = iv.hit_rate
            hit_rate[row.source] = iv.hit_rate
        df.to_csv(csv_path, index=False, mode='w', header=True)

# Add Ratio of weight/(total weight - weight) same for estimates

In [3]:
trials = [5000, 10000, 15000, 20000, 25000, 30000]
sim_id = [1, 2, 3]
path = 'results/'

def add_ratio(row, names, df):
    rows = df.loc[(df.target == row.target) & (df.source != row.source), :]
    for name in names:
        sum_others = rows.loc[:, name].sum()
        row[name + '_ratio'] = row[name] / sum_others
    return row

for sim in sim_id:  
    for trial in trials:
        csv_path = 'results/params_{}_analyse_{}.csv'.format(sim, trial)
        df = pd.read_csv(csv_path)
        df = df.apply(add_ratio, names=['weight', 'iv_wald', 'cch', 'iv_cch'], df=df, axis=1)
        df.to_csv(csv_path, index=False, mode='w', header=True)